In [1]:
import tqdm
import pandas as pd
import os

data_dir = '../public/models/'
info = pd.read_csv(os.path.join(data_dir, 'dataset.csv.gz'), encoding='utf-8')
info['genres'] = info.genres.apply(lambda x: x.strip('[]').split(', '))
print(info.columns)

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index([u'annotation', u'author', u'date', u'flibusta_id', u'genres', u'id',
       u'language', u'program', u'publisher', u'source', u'title', u'year',
       u'self_published_rule_based'],
      dtype='object')


In [2]:
samizdat_publishers = [
    u'си',
    u'selfpub',
    u'самиздат',
    u'издательские решения',
    u'ридеро',
]

In [3]:
info['self_published_rule_based'] = (
    info.genres.apply(lambda genres: 'network_literature' in genres) | 
    info.publisher.fillna('').apply(lambda x: x.strip().lower()).isin(samizdat_publishers) |
    info.publisher.fillna('').apply(lambda x: x.find(u'СИ') != -1) | 
    info.source.fillna('').apply(lambda x: x.find(u'samlib.ru') != -1) |
    info.source.fillna('').apply(lambda x: x.find(u'litmir') != -1) |
    info.source.fillna('').apply(lambda x: x.find(u'proza.ru') != -1)
)

In [4]:
info['publisher'] = info['publisher'].str.strip().str.lower()

In [5]:
import numpy as np
import re

re_ast = re.compile(ur'(\bаст\b)|(\bast\b)', re.UNICODE)
info['publisher'] = info['publisher'].fillna('')
info['publisher_eksmo'] = info.publisher.apply(lambda x: x.find(u'эксмо') != -1)
info['publisher_ast'] = info.publisher.apply(lambda x: bool(re_ast.search(x)))
info['publisher_si'] = info.publisher.apply(lambda x: (x.find(u'си') != -1) | (x.find(u'самиздат') != -1))
info['publisher_mg'] = info.publisher.apply(lambda x: (x.find(u'молодая гвардия') != -1))
info['publisher_azbuka'] = info.publisher.apply(lambda x: (x.find(u'азбука') != -1))
info['publisher_cp'] = info.publisher.apply(lambda x: (x.find(u'центрполиграф') != -1))
info['publisher_dl'] = info.publisher.apply(lambda x: (x.find(u'детская литература') != -1))
info['publisher_veche'] = info.publisher.apply(lambda x: (x.find(u'вече') != -1))
info['publisher_alpha'] = info.publisher.apply(lambda x: (x.find(u'альфа') != -1))
columns = [c for c in info.columns if c.startswith('publisher')]
sample = info[info.publisher.astype(bool) & (info.language == 'ru') & info[columns[1:]].apply(np.any, axis = 1)]

In [6]:
sample[['author', 'annotation', 'title'] + columns[1:]].to_csv('publisher_sample.csv', index=False, encoding='utf-8')

In [7]:
from itertools import chain
from collections import Counter

genres = [g for g, count in Counter(chain(*info.genres.values)).most_common(20) if g!='network_literature']

for g in genres:
    info['genre_%s' % g] = info.genres.apply(lambda genres: g in genres)

In [8]:
genres

[u'sf',
 u'prose_contemporary',
 u'sf_fantasy',
 u'nonf_biography',
 u'sci_history',
 u'nonf_publicism',
 u'sf_action',
 u'detective',
 u'prose_classic',
 u'love_contemporary',
 u'love_short',
 u'love_sf',
 u'sf_horror',
 u'love_history',
 u'sf_history',
 u'prose_rus_classic',
 u'prose_history',
 u'prose_su_classics',
 u'child_prose']

In [9]:
import numpy as np

genre_columns = ['genre_%s' % g for g in genres]
sample_genres = info[(info[genre_columns].apply(np.sum, axis=1) == 1) & (info.language == 'ru')]

In [ ]:
sample_genres[['author', 'annotation', 'title'] + genre_columns].to_csv('genre_sample.csv', index=False, encoding='utf-8')